# Preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl ; mpl.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt ; plt.rcParams['font.family'] = 'AppleGothic'
import seaborn as sns

---

In [2]:
# Train Data 불러오기 및 오류 데이터 제거
train = pd.read_csv('train.csv')
train_error = ['C2085', 'C1397', 'C2431', 'C1649', 'C1036', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
train = train[~train['단지코드'].isin(train_error)].reset_index(drop=True)
print(train.shape)

(2869, 15)


In [3]:
# Test Data 불러오기 및 오류 데이터 제거
test = pd.read_csv('test.csv')
test_error = ['C2675', 'C2335', 'C1327']
test = test[~test['단지코드'].isin(test_error)].reset_index(drop=True)
print(test.shape)

(1008, 14)


---

In [4]:
print(train.loc[train['임대건물구분'] == '상가', '전용면적별세대수'].unique())

[1]


$\Rightarrow$ 임대건물구분이 '상가'인 경우는 무조건 한 세대

In [5]:
# Train Data의 '상가' 변수 생성
train['상가'] = train['단지코드'].isin(train.loc[train['임대건물구분'] == '상가', '단지코드'].unique()).astype('int64')

In [6]:
# Test Data의 '상가' 변수 생성
test['상가'] = test['단지코드'].isin(test.loc[train['임대건물구분'] == '상가', '단지코드'].unique()).astype('int64')

In [7]:
# Train Data의 '임대건물구분 == 상가' 데이터 제거, '임대건물구분' 변수 제거
train = train[train['임대건물구분'] != '상가'].reset_index(drop=True)
train.drop(['임대건물구분'], axis=1, inplace=True)
print(train.shape)

(2307, 15)


In [8]:
# Test Data의 '임대건물구분 == 상가' 데이터 제거, '임대건물구분' 변수 제거
test = test[test['임대건물구분'] != '상가'].reset_index(drop=True)
test.drop(['임대건물구분'], axis=1, inplace=True)
print(test.shape)

(831, 14)


---

In [9]:
# Train Data의 '전용면적별세대수합' 변수 생성
noh_by_area = train.groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합'})
train = pd.merge(left=train, right=noh_by_area, how='left', on='단지코드')
print(train.shape)

(2307, 16)


In [10]:
# Test Data의 '전용면적별세대수합' 변수 생성
noh_by_area = test.groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합'})
test = pd.merge(left=test, right=noh_by_area, how='left', on='단지코드')
print(test.shape)

(831, 15)


---

In [11]:
print(set(train['지역'].unique()) - set(test['지역'].unique()))
print(set(train['자격유형'].unique()) - set(test['자격유형'].unique()))
print(set(train['공급유형'].unique()) - set(test['공급유형'].unique()))

{'서울특별시'}
{'B', 'O', 'F'}
{'장기전세', '공공임대(5년)', '공공분양'}


In [12]:
# Test Data에 없는 카테고리 Train Data에서 제거
train = train[train['지역'] != '서울특별시'].reset_index(drop=True)
train = train[~train['자격유형'].isin(['F', 'O', 'B'])].reset_index(drop=True)
train = train[~train['공급유형'].isin(['공공임대(5년)', '장기전세', '공공분양'])].reset_index(drop=True)
print(train.shape)

(2222, 16)


---

In [13]:
# Train Data의 '임대보증금', '임대료'의 '-'를 0으로 변환
train['임대보증금'] = train['임대보증금'].replace('-', 0).astype('float64')
train['임대료'] = train['임대료'].replace('-', 0).astype('float64')

In [14]:
# Test Data의 '임대보증금', '임대료'의 '-'를 0으로 변환
test['임대보증금'] = test['임대보증금'].replace('-', 0).astype('float64')
test['임대료'] = test['임대료'].replace('-', 0).astype('float64')

---

In [15]:
# Train Data의 '도보 10분거리 내 지하철역 수(환승노선 수 반영)' NA 0으로 대체
train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].fillna(0)

In [16]:
# Test Data의 '도보 10분거리 내 지하철역 수(환승노선 수 반영)' NA 0으로 대체
test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].fillna(0)

---

In [17]:
# Test Data의 '단지코드' NA 특정값으로 대체
test.loc[test['단지코드'] == 'C2411', '자격유형'] = 'A'
test.loc[test['단지코드'] == 'C2253', '자격유형'] = 'D'

---

In [18]:
# Test Data의 '임대보증금', '임대료' NA 특정값으로 대체
test.loc[test['임대보증금'].isnull(), '임대보증금'] = [5787000.0, 5787000.0, 11574000.0]
test.loc[test['임대료'].isnull(), '임대료'] = [79980.0, 79980.0, 159960.0]

---

In [19]:
# Train Data의 '자격유형' 변수 재생성 (Major Voting)
qual_type = train.groupby(['단지코드', '자격유형'], as_index=False).agg({'전용면적별세대수':'sum'}).sort_values('전용면적별세대수', ascending=False).drop_duplicates('단지코드').reset_index(drop=True).drop(['전용면적별세대수'], axis=1)
train = pd.merge(train, qual_type, how='left', on='단지코드').drop('자격유형_x', axis=1).rename(columns={'자격유형_y':'자격유형'})

In [20]:
# Train Data의 '공급유형' 변수 재생성 (Major Voting)
supply_type = train.groupby(['단지코드', '공급유형'], as_index=False).agg({'전용면적별세대수':'sum'}).sort_values('전용면적별세대수', ascending=False).drop_duplicates('단지코드').reset_index(drop=True).drop(['전용면적별세대수'], axis=1)
train = pd.merge(train, supply_type, how='left', on='단지코드').drop('공급유형_x', axis=1).rename(columns={'공급유형_y':'공급유형'})

---

In [21]:
# Test Data의 '자격유형' 변수 재생성 (Major Voting)
qual_type = test.groupby(['단지코드', '자격유형'], as_index=False).agg({'전용면적별세대수':'sum'}).sort_values('전용면적별세대수', ascending=False).drop_duplicates('단지코드').reset_index(drop=True).drop(['전용면적별세대수'], axis=1)
test = pd.merge(test, qual_type, how='left', on='단지코드').drop('자격유형_x', axis=1).rename(columns={'자격유형_y':'자격유형'})

In [22]:
# Test Data의 '공급유형' 변수 재생성 (Major Voting)
supply_type = test.groupby(['단지코드', '공급유형'], as_index=False).agg({'전용면적별세대수':'sum'}).sort_values('전용면적별세대수', ascending=False).drop_duplicates('단지코드').reset_index(drop=True).drop(['전용면적별세대수'], axis=1)
test = pd.merge(test, supply_type, how='left', on='단지코드').drop('공급유형_x', axis=1).rename(columns={'공급유형_y':'공급유형'})

---

In [23]:
# Train Data의 '실세대수'='총세대수'-'공가수' 변수 생성, '총세대수', '공가수' 변수 제거
train['실세대수'] = train['총세대수'] - train['공가수']
train.drop(['총세대수', '공가수'], axis=1, inplace=True)
print(train.shape)

(2222, 15)


In [24]:
# Test Data의 '실세대수'='총세대수'-'공가수' 변수 생성, '총세대수', '공가수' 변수 제거
test['실세대수'] = test['총세대수'] - test['공가수']
test.drop(['총세대수', '공가수'], axis=1, inplace=True)
print(test.shape)

(831, 14)


---

In [25]:
# Train Data의 '임대료' 변수 재생성 (전용면적별세대수 가중 평균)
train['임대료'] = round(train['임대료'] * train['전용면적별세대수'] / train['전용면적별세대수합'])
rental_fee = train.groupby('단지코드', as_index=False).agg({'임대료':'sum'})
train = pd.merge(train, rental_fee, how='left', on='단지코드').drop(['임대료_x'], axis=1).rename(columns={'임대료_y':'임대료'})

In [26]:
# Test Data의 '임대료' 변수 재생성 (전용면적별세대수 가중 평균)
test['임대료'] = round(test['임대료'] * test['전용면적별세대수'] / test['전용면적별세대수합'])
rental_fee = test.groupby('단지코드', as_index=False).agg({'임대료':'sum'})
test = pd.merge(test, rental_fee, how='left', on='단지코드').drop(['임대료_x'], axis=1).rename(columns={'임대료_y':'임대료'})

In [27]:
# Train Data의 '임대보증금' 변수 재생성 (전용면적별세대수 가중 평균)
train['임대보증금'] = round(train['임대보증금'] * train['전용면적별세대수'] / train['전용면적별세대수합'])
rental_fee = train.groupby('단지코드', as_index=False).agg({'임대보증금':'sum'})
train = pd.merge(train, rental_fee, how='left', on='단지코드').drop(['임대보증금_x'], axis=1).rename(columns={'임대보증금_y':'임대보증금'})

In [28]:
# Test Data의 '임대보증금' 변수 재생성 (전용면적별세대수 가중 평균)
test['임대보증금'] = round(test['임대보증금'] * test['전용면적별세대수'] / test['전용면적별세대수합'])
rental_fee = test.groupby('단지코드', as_index=False).agg({'임대보증금':'sum'})
test = pd.merge(test, rental_fee, how='left', on='단지코드').drop(['임대보증금_x'], axis=1).rename(columns={'임대보증금_y':'임대보증금'})

---

In [29]:
# Train Data의 '전용면적' 변수 재생성 (전용면적별세대수 가중 평균)
train['전용면적'] = round(train['전용면적'] * train['전용면적별세대수'] / train['전용면적별세대수합'])
rental_fee = train.groupby('단지코드', as_index=False).agg({'전용면적':'sum'})
train = pd.merge(train, rental_fee, how='left', on='단지코드').drop(['전용면적_x'], axis=1).rename(columns={'전용면적_y':'전용면적'})

In [30]:
# Test Data의 '전용면적' 변수 재생성 (전용면적별세대수 가중 평균)
test['전용면적'] = round(test['전용면적'] * test['전용면적별세대수'] / test['전용면적별세대수합'])
rental_fee = test.groupby('단지코드', as_index=False).agg({'전용면적':'sum'})
test = pd.merge(test, rental_fee, how='left', on='단지코드').drop(['전용면적_x'], axis=1).rename(columns={'전용면적_y':'전용면적'})

---

In [31]:
# Train Data의 '아파트수' 변수 생성
num_of_apt = pd.DataFrame(train['단지코드'].value_counts()).reset_index().rename(columns={'index':'단지코드', '단지코드':'아파트수'})
train = pd.merge(train, num_of_apt, how='left', on='단지코드')
print(train.shape)

(2222, 16)


In [32]:
# Test Data의 '아파트수' 변수 생성
num_of_apt = pd.DataFrame(test['단지코드'].value_counts()).reset_index().rename(columns={'index':'단지코드', '단지코드':'아파트수'})
test = pd.merge(test, num_of_apt, how='left', on='단지코드')
print(test.shape)

(831, 15)


---

In [33]:
# Train Data의 '차량보유인구비율' 변수 생성
age_gender_info = pd.read_csv('age_gender_info.csv')
cars = pd.DataFrame(age_gender_info.iloc[:, [7, 8, 9, 10, 12, 14]].sum(axis=1), columns=['차량보유인구비율'])
cars['지역'] = age_gender_info['지역']
train = pd.merge(train, cars, how='left', on='지역')
print(train.shape)

(2222, 17)


In [34]:
# Test Data의 '차량보유인구비율' 변수 생성
test = pd.merge(test, cars, how='left', on='지역')
print(test.shape)

(831, 16)


---

In [35]:
tmp = train.drop(['전용면적별세대수'], axis=1).drop_duplicates().reset_index(drop=True)

In [36]:
tmp['공급유형'] = [1 if (i == '행복주택') | (i == '영구임대') else 0 for i in tmp['공급유형']]

In [37]:
tmp1 = tmp[tmp['공급유형'] == 1].drop(['공급유형'], axis=1)
tmp2 = tmp[tmp['공급유형'] == 0].drop(['공급유형'], axis=1)

In [38]:
X_train1 = tmp1[['단지내주차면수']]
y_train1 = tmp1['등록차량수']
X_train2 = tmp2[['단지내주차면수', '전용면적', '차량보유인구비율', '실세대수']]
y_train2 = tmp2['등록차량수']

In [39]:
from sklearn.preprocessing import PolynomialFeatures

In [40]:
poly = PolynomialFeatures(2, interaction_only=True)

In [41]:
X_train2 = poly.fit_transform(X_train2)[:, [0, 1, 2, 3, 4, 5, 6, 8, 10]]

In [42]:
print(X_train1.shape, X_train2.shape)

(61, 1) (340, 9)


In [43]:
from sklearn.preprocessing import StandardScaler

In [44]:
scaler1 = StandardScaler()
scaler1.fit(X_train1)
scaler2 = StandardScaler()
scaler2.fit(X_train2)

StandardScaler()

In [45]:
X_train1 = scaler1.transform(X_train1)
X_train2 = scaler2.transform(X_train2)

In [46]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [47]:
lr1 = LinearRegression()
lr2 = LinearRegression()

In [48]:
score1 = cross_val_score(lr1, X_train1, y_train1, cv=10, scoring='neg_mean_absolute_error')
score2 = cross_val_score(lr2, X_train2, y_train2, cv=10, scoring='neg_mean_absolute_error')

In [49]:
print(f'평균: {-score1.mean()} | 표준편차: {score1.std()}')
print(f'평균: {-score2.mean()} | 표준편차: {score2.std()}')

평균: 77.40938954037743 | 표준편차: 27.984961935188633
평균: 123.44197702990289 | 표준편차: 42.13279739181187


In [50]:
print('평균 가중평균:', (-score1.mean()*len(X_train1)+-score2.mean()*len(X_train2))/(len(X_train1)+len(X_train2)))
print('표준편차 가중평균:', (score1.std()*len(X_train1)+score2.std()*len(X_train2))/(len(X_train1)+len(X_train2)))

평균 가중평균: 116.43951359633417
표준편차 가중평균: 39.980632895916564


In [51]:
lr1.fit(X_train1, y_train1)
lr2.fit(X_train2, y_train2)

LinearRegression()

---

In [52]:
tmp_test = test.drop(['전용면적별세대수', '임대보증금'], axis=1).drop_duplicates().reset_index(drop=True)

In [53]:
tmp_test['공급유형'] = [1 if (i == '행복주택') | (i == '영구임대') else 0 for i in tmp_test['공급유형']]

In [54]:
tmp_test1 = tmp_test[tmp_test['공급유형'] == 1].drop(['공급유형'], axis=1)
tmp_test2 = tmp_test[tmp_test['공급유형'] == 0].drop(['공급유형'], axis=1)

In [55]:
X_test1 = tmp_test1[['단지내주차면수']]
X_test2 = tmp_test2[['단지내주차면수', '전용면적', '차량보유인구비율', '실세대수']]

In [56]:
poly = PolynomialFeatures(2, interaction_only=True)

In [57]:
X_test2 = poly.fit_transform(X_test2)[:, [0, 1, 2, 3, 4, 5, 6, 8, 10]]

In [58]:
X_test1 = scaler1.transform(X_test1)
X_test2 = scaler2.transform(X_test2)

In [59]:
tmp_test1['등록차량수'] = lr1.predict(X_test1)
tmp_test2['등록차량수'] = lr2.predict(X_test2)

In [60]:
tmp_test = pd.concat([tmp_test1, tmp_test2])

In [61]:
submission = tmp_test[['단지코드', '등록차량수']].rename(columns={'단지코드':'code', '등록차량수':'num'})

In [62]:
error = pd.DataFrame(np.array([['C2675', 0], ['C2335', 0], ['C1327', 0]]), columns=['code', 'num'])

In [63]:
submission = pd.concat([submission, error]).reset_index(drop=True)

In [64]:
submission.to_csv('submission.csv', index=False)